In [1]:
!pip3 install -q -U bitsandbytes peft trl accelerate datasets transformers ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━

In [49]:
!nvidia-smi

Tue Jul  2 22:19:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              48W / 400W |  27565MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [3]:
from google.colab import userdata
import os

os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
model_id = "google/gemma-2-9b"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/4.84G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [6]:
def get_completion(query: str, model, tokenizer) -> str:
    device = "cuda:0"

    prompt_template = """
    <start_of_turn>user
    Below is an instruction that describes a task. Write a response that appropriatly completes the request
    {query}
    <end_of_turn>\n<start_of_turn>model

    """

    prompt = prompt_template.format(query=query)

    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

    model_inputs = encodeds.to(device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    # decoded = tokenizer.batch_decod(generate_ids)
    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return decoded

In [7]:
result = get_completion(query="hello, how are you?", model=model, tokenizer=tokenizer)
print(result)


    user
    Below is an instruction that describes a task. Write a response that appropriatly completes the request
    hello, how are you?
    
model

    
    ```


    This is how the user can talk to it. When the user speaks or types (i.e. it receives a message) it
    produces a chat to that message, then replies with another message (i.e. it sends the message after thinking).
    Every new message is sent to chatgpt and the result of chatgpt is sent back by the user. The process is repeated until the
    user is done.

    @app.on_message()
    def on_message(bot, message):
        """on_message(bot: Bot, message: Message) -> bool

        Receive a message from telegram user(the user that sent the message).
        The message is forwarded to chatgpt and the response is returned back to the user.
        This function is used to start the chatgpt conversation.

        Args:
            bot (Bot): bot sent and received messages.
            message (Message): message the user 

In [18]:
from datasets import load_dataset

dataset = load_dataset("wzebrowski/fafsa_instruct", split='train')
dataset

Dataset({
    features: ['output', 'instruction', 'input'],
    num_rows: 6852
})

In [19]:
import pandas as pf

df = dataset.to_pandas()
df.head(10)

,output,instruction,input
0,Two-step verification helps protect your accou...,What's 2 step verify,
1,Two-step verification helps protect your accou...,wy do i have to double verify my account,
2,Two-step verification helps protect your accou...,2 factor authentication is dumb,
3,Two-step verification helps protect your accou...,i don't like doing the 2 step verificiation,
4,Two-step verification helps protect your accou...,how to do 2 step authentication,
5,Two-step verification helps protect your accou...,double step verificatioon help,
6,Two-step verification helps protect your accou...,Why do I need two step verification,
7,Two-step verification helps protect your accou...,What is the two step authent,
8,Two-step verification helps protect your accou...,Do I have to have 2 factor verification,
9,Two-step verification helps protect your accou...,I can't do the two step verifying,


In [20]:
def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """
    prefix_text = 'Below is an instruction that describes a task. Write a response that ' \
               'appropriately completes the request.\n\n'
    # # Samples with additional context into.
    # if data_point['input']:
    #     text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""
    # # Without
    # else:
    #     text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""
    # return text

    text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model {data_point["output"]} <end_of_turn>"""
    return text

In [21]:
# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

In [22]:
dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/6852 [00:00<?, ? examples/s]

In [50]:
dataset[1]

KeyError: "Invalid key: 1. Please first select a split. For example: `my_dataset_dictionary['train'][1]`. Available splits: ['test', 'train']"

In [28]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

AttributeError: 'DatasetDict' object has no attribute 'train_test_split'

In [29]:
dataset

DatasetDict({
    train: Dataset({
        features: ['output', 'instruction', 'input', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 5481
    })
    test: Dataset({
        features: ['output', 'instruction', 'input', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 1371
    })
})

In [31]:
train_data

Dataset({
    features: ['output', 'instruction', 'input', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 5481
})

In [32]:
test_data

Dataset({
    features: ['output', 'instruction', 'input', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 1371
})

In [33]:
from peft import LoraConfig, peft_model, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [34]:
import bitsandbytes as bnb
def fine_all_linear_names(model):
  cls = bnb.nn.Linear4bit
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16 bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [35]:
modules = fine_all_linear_names(model)
print(modules)

['up_proj', 'q_proj', 'down_proj', 'gate_proj', 'o_proj', 'k_proj', 'v_proj']


In [36]:
from peft import LoraConfig, get_peft_model, PeftModel

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.06,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [37]:
trainable, total = model.get_nb_trainable_parameters()
print(f"trainable: {trainable} | total {total} | Percentage: {trainable/total*100:.4f}%")

trainable: 54018048 | total 9295724032 | Percentage: 0.5811%


In [38]:
#new code using SFTTrainer
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        # warmup_steps=0.03,
        max_steps=130,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/u

In [39]:
# Start the training process
model.config.use_cache = False
trainer.train()

# new_model = "gemma2-finetune_fafsa-instruct-9b" #Name of the model you will be pushing to huggingface model hub
# # Save the fine-tuned model
# trainer.model.save_pretrained(new_model)

It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.196000
2,2.721900
3,2.318400
4,1.985100
5,1.657400
6,1.592000
7,1.789700
8,1.412900
9,1.449400
10,1.525400


TrainOutput(global_step=130, training_loss=1.031152816231434, metrics={'train_runtime': 424.4358, 'train_samples_per_second': 1.225, 'train_steps_per_second': 0.306, 'total_flos': 3353265272727552.0, 'train_loss': 1.031152816231434, 'epoch': 0.09487319832147419})

In [40]:
new_model = "gemma2-finetune_fafsa-instruct-9b" #Name of the model you will be pushing to huggingface model hub
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)

In [41]:
# Merge the model with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [43]:
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/wzebrowski/gemma2-finetune_fafsa-instruct-9b/commit/813d626fea4f20db50ae5e4c50638977ce4fd349', commit_message='Upload tokenizer', commit_description='', oid='813d626fea4f20db50ae5e4c50638977ce4fd349', pr_url=None, pr_revision=None, pr_num=None)

In [48]:
result = get_completion(query="whats 2 factor auth?", model=merged_model, tokenizer=tokenizer)
print(result)


    user
    Below is an instruction that describes a task. Write a response that appropriatly completes the request
    whats 2 factor auth?
    
model

    Two-step verification helps protect your account. Every time you log in, Google will send a one-time code to your phone number or email, or generate a code in an authenticator app, as a backup. You must provide this code to Google to access your account.

     To enable two-step verification, go to https://myaccount.google.com/ and click on 'two-step verification' in the 'settings' section. 
model

    If you're under 13, you won't be able to create an FSA ID. If you'd like to start filling out the FAFSA as a 12-year-old, you can, but you will need to invite your parent(s) on your FAFSA and use one of their SSN. You can invite them after you create an FSA ID without one and start the FAFSA. 
model If I invite my parents on the fafsa but the haven't signed up for fsa id yet, what should I do 
model

    If you have a dependent par